In [1]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds, AllChem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import pandas as pd

from strain_relief import compute_strain

In [2]:
! uv pip install py3Dmol
import py3Dmol

Using Python 3.12.3 environment at: /Users/wallace5/StrainRelief/.venv
Audited 1 package in 1ms


## Running StrainRelief 

In [3]:
# First, lets generate some example poses with 3D coordinates.
smiles = ["CCO", "CCN", "CCC"]
poses = []

for s in smiles:
    mol = Chem.MolFromSmiles(s)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    mol.SetProp("smiles", s)
    poses.append(mol)

# Note: to run StrainRelief your molecules must either have specified 
# bonds or they must be able to be passed through RDKit's rdDetermineBonds 
# function. This is needed for the conformer enumeration.

In [4]:
# We now need to initialise a run configuration.
# We use MMFF94s here so that runs will only take a few seconds.
with initialize(version_base="1.1", config_path="../src/strain_relief/hydra_config"):
    cfg = compose(
        config_name="default", 
        overrides=["experiment=mmff94s",]
    )

print(OmegaConf.to_yaml(cfg))

seed: -1
threshold: 16.1
numThreads: 0
io:
  input:
    parquet_path: ???
    mol_col_name: null
    id_col_name: null
  output:
    parquet_path: null
    mol_col_name: ${..input.mol_col_name}
    id_col_name: ${..input.id_col_name}
conformers:
  randomSeed: ${seed}
  numConfs: 20
  maxAttempts: 10
  pruneRmsThresh: 0.1
  clearConfs: false
  numThreads: ${numThreads}
local_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.5
  fexit: 250
global_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.05
  fexit: 250
energy_eval:
  method: ${global_min.method}



StrainRelief is run either via the `compute_strain` function or via the command line with the `strain-relief` command. The following three examples demonstrate different ways of running the tool, all giving the same output.

In [5]:
# EXAMPLE 1

# The minimal requirement to run StrainRelief is a list of RDKit.Mols with 3D poses and a run configration.
# If ids are not given then they will be generated.
results = compute_strain(mols=poses, ids=None, cfg=cfg)
results.head()

2025-09-24 23:37:59.499 | INFO     | strain_relief.compute_strain:compute_strain:56 - STRAIN RELIEF RUN CONFIG: 
seed: -1
threshold: 16.1
numThreads: 0
io:
  input:
    parquet_path: ???
    mol_col_name: null
    id_col_name: null
  output:
    parquet_path: null
    mol_col_name: ${..input.mol_col_name}
    id_col_name: ${..input.id_col_name}
conformers:
  randomSeed: ${seed}
  numConfs: 20
  maxAttempts: 10
  pruneRmsThresh: 0.1
  clearConfs: false
  numThreads: ${numThreads}
local_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.5
  fexit: 250
global_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.05
  fexit: 250
energy_eval:
  method: ${global_min.method}

2025-09-24 23:37:59.506 | INFO     | strain_relief.compute_strain:compute_strain:73 - Minimising docked conformer...
2025-09-24 23:37:59.506 | INFO

,id,mol_bytes,formal_charge,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter,nconfs_converged
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,3.363565,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-1.209456,4.573021,True,1
1,1,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-5.219026,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-5.683207,0.464181,True,1
2,2,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,2.182454,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-4.883026,7.065480,True,1


In [6]:
# EXAMPLE 2

# Instead of a list of mols, StrainRelief can be passed a dataframe with a "mol_bytes" and "id" column.
# "mol_bytes" is a column of RDKit.Mol objects stroed as bytes.
# Any additional columns, such as "smiles" here are also returned.

df = pd.DataFrame([{"mol_bytes": mol.ToBinary(), **mol.GetPropsAsDict()} for mol in poses])
df = df.reset_index(drop=False, names='id')

results2 = compute_strain(df=df, cfg=cfg)
results2.head()

2025-09-24 23:38:00.363 | INFO     | strain_relief.compute_strain:compute_strain:56 - STRAIN RELIEF RUN CONFIG: 
seed: -1
threshold: 16.1
numThreads: 0
io:
  input:
    parquet_path: ???
    mol_col_name: null
    id_col_name: null
  output:
    parquet_path: null
    mol_col_name: ${..input.mol_col_name}
    id_col_name: ${..input.id_col_name}
conformers:
  randomSeed: ${seed}
  numConfs: 20
  maxAttempts: 10
  pruneRmsThresh: 0.1
  clearConfs: false
  numThreads: ${numThreads}
local_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.5
  fexit: 250
global_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProperties:
    mmffVariant: ${..method}
  MMFFGetMoleculeForceField: {}
  fmax: 0.05
  fexit: 250
energy_eval:
  method: ${global_min.method}

2025-09-24 23:38:00.366 | INFO     | strain_relief.compute_strain:compute_strain:73 - Minimising docked conformer...
2025-09-24 23:38:00.366 | INFO

,id,mol_bytes,smiles,formal_charge,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter,nconfs_converged
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,CCO,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,3.363565,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-1.209456,4.573021,True,1
1,1,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",CCN,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-5.219026,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-5.683207,0.464181,True,1
2,2,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,CCC,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,2.182454,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-4.883026,7.065480,True,1


In [7]:
# EXAMPLE 3

# Finally, StrainRelief can be run from the command line by specifying an input parquet path
# There are more example scripts in StrainRelief/examples/
df.to_parquet("../data/tutorial_example.parquet")
! strain-relief io.input.parquet_path=../data/tutorial_example.parquet io.output.parquet_path=../data/tutorial_output.parquet experiment=mmff94s

results3 = pd.read_parquet("../data/tutorial_output.parquet")
results3.head()

2025-09-24 23:38:02.332 | INFO     | strain_relief.io._io:load_parquet:35 - Loading data...
2025-09-24 23:38:02.397 | INFO     | strain_relief.io._io:load_parquet:37 - Loaded 3 posed molecules
2025-09-24 23:38:02.397 | INFO     | strain_relief.io._io:_check_columns:91 - RDKit.Mol column is 'mol'
2025-09-24 23:38:02.398 | INFO     | strain_relief.io._io:_check_columns:97 - ID column is 'id'
2025-09-24 23:38:02.398 | INFO     | strain_relief.compute_strain:compute_strain:56 - STRAIN RELIEF RUN CONFIG: 
seed: -1
threshold: 16.1
numThreads: 0
io:
  input:
    parquet_path: ../data/tutorial_example.parquet
    mol_col_name: null
    id_col_name: null
  output:
    parquet_path: ../data/tutorial_output.parquet
    mol_col_name: ${..input.mol_col_name}
    id_col_name: ${..input.id_col_name}
conformers:
  randomSeed: ${seed}
  numConfs: 20
  maxAttempts: 10
  pruneRmsThresh: 0.1
  clearConfs: false
  numThreads: ${numThreads}
local_min:
  method: MMFF94s
  maxIters: 1000
  MMFFGetMoleculeProp

,id,mol_bytes,smiles,formal_charge,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter,nconfs_converged
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,CCO,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,3.363565,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-1.209456,4.573021,True,1
1,1,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",CCN,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-5.219026,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-5.683207,0.464181,True,1
2,2,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,CCC,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,2.182454,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-4.883026,7.065480,True,1


## Examining the Output

In [8]:
! strain-relief io.input.parquet_path=../data/example_ligboundconf_input.parquet io.output.parquet_path=../data/example_ligboundconf_output.parquet experiment=mmff94s
lig = pd.read_parquet("../data/example_ligboundconf_output.parquet")
lig.head()

2025-09-24 23:38:05.110 | INFO     | strain_relief.io._io:load_parquet:35 - Loading data...
2025-09-24 23:38:05.176 | INFO     | strain_relief.io._io:load_parquet:37 - Loaded 2 posed molecules
2025-09-24 23:38:05.176 | INFO     | strain_relief.io._io:_check_columns:91 - RDKit.Mol column is 'mol'
2025-09-24 23:38:05.176 | INFO     | strain_relief.io._io:_check_columns:97 - ID column is 'id'
2025-09-24 23:38:05.177 | INFO     | strain_relief.compute_strain:compute_strain:56 - STRAIN RELIEF RUN CONFIG: 
seed: -1
threshold: 16.1
numThreads: 0
io:
  input:
    parquet_path: ../data/example_ligboundconf_input.parquet
    mol_col_name: null
    id_col_name: null
  output:
    parquet_path: ../data/example_ligboundconf_output.parquet
    mol_col_name: ${..input.mol_col_name}
    id_col_name: ${..input.id_col_name}
conformers:
  randomSeed: ${seed}
  numConfs: 20
  maxAttempts: 10
  pruneRmsThresh: 0.1
  clearConfs: false
  numThreads: ${numThreads}
local_min:
  method: MMFF94s
  maxIters: 1000

,id,mol_bytes,ligand_id,some_property,formal_charge,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter,nconfs_converged
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,3Q4_3QD0_A_370,A,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-122.361323,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-132.501279,10.139956,True,20
1,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,YTW_6IBK_A_525,B,0,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-14.682596,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-44.309465,29.626869,False,21


The `lig` dataframe contains all input columns (in this case `id`, `mol_bytes` and `some_property`) and all calculated columns:
- `formal_charge` (int): RDKit's formal charge
- `local_min_mol` (bytes): the coordinates of the local minimum
- `local_min_e` (float): the energy of the local minimum (in kcal/mol)
- `global_min_mol` (bytes): the coordinates of the global minimum
- `global_min_e` (float): the energy of the global minimum (in kcal/mol)
- `ligand_strain` (float): difference between local and global minima
- `passes_strain_filter` (bool): whether `ligand_strain` is lower than the config threshold
- `nconfs_converged` (int): the number of conformers that convereged when searching for the global minimum 

Lets have a look at the three poses from ligand 3Q4_3QD0_A_370. 

In [9]:
docked = Chem.Mol(lig.mol_bytes[0])
local_min = Chem.Mol(lig.local_min_mol[0])
global_min = Chem.Mol(lig.global_min_mol[0])

In [10]:
rdDetermineBonds.DetermineBonds(docked)
rdDetermineBonds.DetermineBonds(local_min)
rdDetermineBonds.DetermineBonds(global_min)

In [11]:
IPythonConsole.drawMol3D(docked)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [12]:
IPythonConsole.drawMol3D(local_min)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [13]:
IPythonConsole.drawMol3D(global_min)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The original and local minimum conformers look very similar to the eye. This is because local minimisation has a loose convergence criteria and is simply to clean up any high energy artifacts left by docking. The global minimum is noticably different, with all aromatic rings having relaxed into a similar plane.

You may again want to convert your results back into an sdf. You can do this with the function below:

Hopefully you now have a good grasp on how to run the StrainRelief tool! I hope you find it as useful as we have.

Ewan